In [ ]:
## A1 Your caffeine level

In [ ]:
r.reset(roadrunner.SelectionRecord.ALL)

# set the oral dose
# r['init(PODOSE_caf)'] = 100
r['init(IVDOSE_caf)'] = 100  # [mg]

# simulate the model for 24[h]
s = simulate(r, start=0, end=24, steps=500)

# show the resulting DataFrame

# plot venous caffeine & paraxanthine against time
f1, ax1 = plt.subplots(1, 1, figsize=(5, 5))
ax1.plot(s.time, s.Cve_caf, linewidth=2, label="caffeine [mg/l]")
ax1.plot(s.time, s.Cve_px, linewidth=2, label="paraxanthine [mg/l]")
ax1.set_title('100 [mg] caffeine (oral dose)')
ax1.set_ylabel('concentration')
ax1.set_xlabel('time [h]')
ax1.legend()
plt.show()